In [1]:
# bus 1 #step of Data Extract for Kadamba Transport Corporation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_ktclroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            ktcl_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            ktcl_urls = [element.get_attribute("href") for element in ktcl_elements]
            all_ktclroute_urls.extend(ktcl_urls)
            
            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_ktclroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary

            except Exception as e:
               print(f"Failed to click expand button: {e}")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            try:    
                button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
                button.click()
                time.sleep(2)
            except Exception as e:
                print(f"Failed to click expand button: {e}") 
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    df = pd.DataFrame(bus_details)
    print(df)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Failed to navigate to page 5: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//div[contains(@class, "DC_117_pageTabs") and text()="5"]"}
  (Session info: chrome=126.0.6478.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E110BBAC]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E110BA76]
	(No symbol) [0x00007FF7E113D240]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier 

In [32]:
df

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[Kadamba Transport Corporation Limited (KTCL) ...,[Non AC Seater 2+2],[19:00],[02:30],[07h 30m],[420],[24 Seats available\n9 Window],[3.9]
1,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[IntrCity SmartBus],[AC Sleeper (2+1)],[16:30],[22:30],[06h 00m],[550],[9 Seats available\n4 Single],[4.1]
2,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[IntrCity SmartBus],[AC Sleeper (2+1)],[18:20],[00:35],[06h 15m],[600],[16 Seats available\n4 Single],[4.3]
3,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[IntrCity SmartBus],[AC Sleeper (2+1)],[18:30],[00:45],[06h 15m],[599],[13 Seats available\n3 Single],[4.5]
4,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[Amrutyog Travels],[A/C Sleeper (2+1)],[19:00],[02:30],[07h 30m],[555],[20 Seats available\n4 Single],[4.4]
...,...,...,...,...,...,...,...,...,...,...
127,Calangute (goa) to Goa Bus,https://www.redbus.in/bus-tickets/calangute-go...,[Kadamba Transport Corporation Limited (KTCL) ...,[AC Seater (2+2)],[15:30],[16:01],[00h 31m],[100],[41 Seats available\n19 Window],[3.7]
128,Calangute (goa) to Panaji Bus,https://www.redbus.in/bus-tickets/calangute-go...,[Kadamba Transport Corporation Limited (KTCL) ...,[Volvo AC Seater 2+2],[13:30],[14:00],[00h 30m],[100],[38 Seats available\n18 Window],[3.7]
129,Calangute (goa) to Panaji Bus,https://www.redbus.in/bus-tickets/calangute-go...,[Kadamba Transport Corporation Limited (KTCL) ...,[AC Seater (2+2)],[15:30],[16:00],[00h 30m],[100],[41 Seats available\n19 Window],[3.7]
130,Goa Airport to Panaji Bus,https://www.redbus.in/bus-tickets/goa-airport-...,[Kadamba Transport Corporation Limited (KTCL) ...,[AC Seater (2+2)],[13:45],[14:45],[01h 00m],[150],[41 Seats available\n19 Window],[4.4]


In [ ]:
pip install pandas

In [35]:
# bus-2 #Extract Data of HRTC(Himachal Road Transport Corporation)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_hrtcroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            hrtc_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            hrtc_urls = [element.get_attribute("href") for element in hrtc_elements]
            all_hrtcroute_urls.extend(hrtc_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_hrtcroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary

            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    hrtc = pd.DataFrame(bus_details)
    print(hrtc)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Chandigarh to Hamirpur (Himachal Pradesh) B

In [36]:
hrtc

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1516],[A/C Executive (2+3)],[15:00],[18:50],[03h 50m],[315],[44 Seats available\n16 Window],[4.4]
1,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1546],[Ordinary 3+2 Non AC Seater],[16:25],[21:15],[04h 50m],[309],[24 Seats available\n7 Window],[4.8]
2,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 9235],[A/C Executive (2+3)],[16:40],[22:00],[05h 20m],[369],[44 Seats available\n16 Window],[3.9]
3,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 401670],[Ordinary 3+2 Non AC Seater],[18:00],[22:00],[04h 00m],[295],[37 Seats available\n13 Window],[5.0]
4,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1549],[Ordinary 3+2 Non AC Seater],[18:15],[01:00],[06h 45m],[337],[37 Seats available\n13 Window],[3.9]
...,...,...,...,...,...,...,...,...,...,...
538,Delhi to Sarkaghat Bus,https://www.redbus.in/bus-tickets/delhi-to-sar...,[HRTC - 534],[Ordinary 3+2 Non AC Seater],[18:05],[05:20],[11h 15m],[612],[37 Seats available\n13 Window],[4.5]
539,Delhi to Sarkaghat Bus,https://www.redbus.in/bus-tickets/delhi-to-sar...,[HRTC - 541],[Ordinary 3+2 Non AC Seater],[19:14],[04:10],[08h 56m],[575],[37 Seats available\n13 Window],[5.0]
540,Delhi to Sarkaghat Bus,https://www.redbus.in/bus-tickets/delhi-to-sar...,[HRTC - 549],[A/C Executive (2+3)],[19:25],[06:45],[11h 20m],[701],[36 Seats available\n10 Window],[2.7]
541,Delhi to Sarkaghat Bus,https://www.redbus.in/bus-tickets/delhi-to-sar...,[HRTC - 557],[Ordinary 3+2 Non AC Seater],[20:15],[07:10],[10h 55m],[575],[31 Seats available\n11 Window],[3.8]


In [37]:
#bus-3 Extract Data from WBTC West Bengal
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_wbtcroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            wbtc_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            wbtc_urls = [element.get_attribute("href") for element in wbtc_elements]
            all_wbtcroute_urls.extend(wbtc_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_wbtcroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    wbtc = pd.DataFrame(bus_details)
    print(wbtc)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Barasat (West Bengal) to Nandakumar (west b

In [38]:
wbtc

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Barasat (West Bengal) to Nandakumar (west beng...,https://www.redbus.in/bus-tickets/barasat-west...,[WBTC (CTC) BARASAT-DIGHA via Bally - 38|15:00],[Non AC Seater (2+3)],[15:00],[16:45],[01h 45m],[90],[33 Seats available\n13 Window],[3.3]
1,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KOLKATA - A/C - 14:20 (HWH DE...,[AC Seater (2+2)],[14:20],[18:50],[04h 30m],[360],[35 Seats available\n16 Window],[3.2]
2,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KOLKATA - 15:05 (HWH DEPOT) -...,[Non AC Seater (2+3)],[15:05],[19:35],[04h 30m],[180],[39 Seats available\n14 Window],[3.2]
3,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KARUNAMOYEE - 15:10 (BEL DEPO...,[Non AC Seater (2+3)],[15:10],[20:10],[05h 00m],[185],[43 Seats available\n12 Window],[3.5]
4,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KOLKATA - 15:35 (HOWRAH DEPOT...,[Non AC Seater (2+3)],[15:35],[20:05],[04h 30m],[180],[51 Seats available\n19 Window],[1.8]
...,...,...,...,...,...,...,...,...,...,...
100,Kolkata to Asansol (West Bengal) Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,[Royal Cruiser],[Volvo A/C Seater (2+2)],[19:30],[00:15],[04h 45m],[],[27 Seats available\n11 Window],[3.9]
101,Kolkata to Asansol (West Bengal) Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,[Beauty Travels],[NON A/C Seater / Sleeper (2+2)],[20:20],[00:30],[04h 10m],[380],[36 Seats available\n18 Window],[3.0]
102,Kolkata to Asansol (West Bengal) Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,[Pariwartan Bus Service],[NON A/C Seater / Sleeper (2+2)],[18:30],[00:30],[06h 00m],[500],[54 Seats available\n26 Window],[]
103,Kolkata to Asansol (West Bengal) Bus,https://www.redbus.in/bus-tickets/kolkata-to-a...,[Maharani Travels],[A/C Seater / Sleeper (2+1)],[21:05],[02:00],[04h 55m],[],[24 Seats available\n6 Single],[2.6]


In [39]:
#bus-4 Extract Data from CTU RTC Chandigarh Transport Undertaking (CTU)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_cturoute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            ctu_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            ctu_urls = [element.get_attribute("href") for element in ctu_elements]
            all_cturoute_urls.extend(ctu_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_cturoute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    ctu = pd.DataFrame(bus_details)
    print(ctu)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Chandigarh to Hamirpur (Himachal Pradesh) B

In [40]:
ctu

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1546],[Ordinary 3+2 Non AC Seater],[16:25],[21:15],[04h 50m],[309],[24 Seats available\n7 Window],[4.8]
1,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 9235],[A/C Executive (2+3)],[16:40],[22:00],[05h 20m],[369],[44 Seats available\n16 Window],[3.9]
2,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 401670],[Ordinary 3+2 Non AC Seater],[18:00],[22:00],[04h 00m],[295],[37 Seats available\n13 Window],[5.0]
3,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1549],[Ordinary 3+2 Non AC Seater],[18:15],[01:00],[06h 45m],[337],[37 Seats available\n13 Window],[3.9]
4,Chandigarh to Hamirpur (Himachal Pradesh) Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1227],[Ordinary 3+2 Non AC Seater],[19:30],[03:00],[07h 30m],[339],[37 Seats available\n13 Window],[3.9]
...,...,...,...,...,...,...,...,...,...,...
500,Baijnath to Chandigarh Bus,https://www.redbus.in/bus-tickets/baijnath-to-...,[Bharmani Travels],[Volvo A/C Semi Sleeper (2+2)],[20:30],[02:30],[06h 00m],[],[18 Seats available\n8 Window],[3.9]
501,Narnaul to Chandigarh Bus,https://www.redbus.in/bus-tickets/narnaul-to-c...,[Chandigarh Transport Undertaking (CTU) - 165855],[HVAC Seater (2+3)],[17:00],[23:00],[06h 00m],[533.34],[46 Seats available\n17 Window],[4.6]
502,Narnaul to Chandigarh Bus,https://www.redbus.in/bus-tickets/narnaul-to-c...,[Hari Das Tour &Travels],[Bharat Benz A/C Seater /Sleeper (2+1)],[22:15],[04:20],[06h 05m],[],[25 Seats available],[3.6]
503,Chandigarh to Jawala Ji Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[HRTC - 1514],[Ordinary 3+2 Non AC Seater],[16:00],[21:45],[05h 45m],[318],[33 Seats available\n12 Window],[3.9]


In [41]:
#bus-5 Extract Data from PEPSU Punjab
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_pepsuroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            pepsu_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            pepsu_urls = [element.get_attribute("href") for element in pepsu_elements]
            all_pepsuroute_urls.extend(pepsu_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_pepsuroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url, 
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    pepsu= pd.DataFrame(bus_details)
    print(pepsu)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Failed to navigate to page 4: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Phagwara to Delhi Airport Bus
success
Extracted Route Name: Delhi Airport to Ludhia

In [42]:
pepsu

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Phagwara to Delhi Airport Bus,https://www.redbus.in/bus-tickets/phagwara-to-...,[PEPSU (Punjab) - 67098],[Volvo AC Seater (2+2)],[18:35],[04:05],[09h 30m],[1078],[36 Seats available\n18 Window],[4.0]
1,Phagwara to Delhi Airport Bus,https://www.redbus.in/bus-tickets/phagwara-to-...,[PEPSU (Punjab) - 67128],[Volvo AC Seater (2+2)],[19:30],[04:50],[09h 20m],[1078],[34 Seats available\n14 Window],[4.8]
2,Phagwara to Delhi Airport Bus,https://www.redbus.in/bus-tickets/phagwara-to-...,[PEPSU (Punjab) - 67164],[Volvo AC Seater (2+2)],[20:20],[05:50],[09h 30m],[1078],[37 Seats available\n18 Window],[4.2]
3,Phagwara to Delhi Airport Bus,https://www.redbus.in/bus-tickets/phagwara-to-...,[Indo Canadian Transport Pvt. Ltd.],[A/C Sleeper (2+1)],[14:35],[23:00],[08h 25m],[3000],[42 Seats available\n28 Window],[4.5]
4,Phagwara to Delhi Airport Bus,https://www.redbus.in/bus-tickets/phagwara-to-...,[Indo Canadian Transport Pvt. Ltd.],[A/C Semi Sleeper (2+2)],[16:35],[03:00],[10h 25m],[3000],[44 Seats available\n22 Window],[4.5]
...,...,...,...,...,...,...,...,...,...,...
193,Chandigarh to Bathinda Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Rajpreet Travels and cargo],[A/C Sleeper (2+1)],[21:00],[01:30],[04h 30m],[905],[34 Seats available],[3.5]
194,Chandigarh to Faridkot Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Jujhar Travels],[Volvo A/C Semi Sleeper (2+2)],[16:20],[20:45],[04h 25m],[600],[36 Seats available\n16 Window],[4.1]
195,Chandigarh to Patiala Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Orbit Aviation Pvt. Ltd.],[A/C Seater (2+2)],[15:25],[16:45],[01h 20m],[200],[43 Seats available\n20 Window],[3.7]
196,Chandigarh to Patiala Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Orbit Aviation Pvt. Ltd.],[A/C Seater (2+2)],[16:55],[18:30],[01h 35m],[200],[44 Seats available\n22 Window],[2.7]


In [43]:
#bus-6 Extract Data from NBSTC NORTH BENGAL STATE TRANSPORT CORPORATION

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/north-bengal-state-transport-corporation')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_nbsroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            nbs_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            nbs_urls = [element.get_attribute("href") for element in nbs_elements]
            all_nbsroute_urls.extend(nbs_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_nbsroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_lin':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    nbs= pd.DataFrame(bus_details)
    print(nbs)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Cooch Behar (West Bengal) to Kolkata Bus
Fa

In [44]:
nbs

,route_name,route_lin,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[NBSTC-KOL-BLG-R1-19:30],[NON A/C Seater Push Back (2+2)],[19:45],[07:15],[11h 30m],[367],[33 Seats available\n15 Window],[3.8]
1,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Express Line],[Volvo 9600 A/C Seater/Sleeper (2+1)],[19:45],[06:15],[10h 30m],[699],[10 Seats available\n3 Window],[4.6]
2,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Shyamoli Paribahan Pvt Ltd],[Scania Multi-Axle AC Semi Sleeper (2+2)],[20:00],[06:15],[10h 15m],[700],[34 Seats available\n13 Window],[4.0]
3,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Rubi Paribahan ],[Volvo 9600 A/C Seater/Sleeper (2+1)],[20:30],[05:00],[08h 30m],[749],[25 Seats available],[4.0]
4,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Sonar Tori],[A/C Seater / Sleeper (2+2)],[19:45],[05:30],[09h 45m],[],[34 Seats available\n17 Window],[3.2]
...,...,...,...,...,...,...,...,...,...,...
159,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,[NBSTC-RAI-KOL-R1-19:30],[NON A/C Seater Push Back (2+2)],[19:30],[04:15],[08h 45m],[278],[35 Seats available\n17 Window],[3.3]
160,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,[Lokenath Bus Service],[NON A/C Seater / Sleeper (2+2)],[19:45],[04:10],[08h 25m],[500],[11 Seats available],[3.0]
161,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,[Aradhana Bus Service],[NON A/C Seater / Sleeper (2+2)],[19:15],[03:10],[07h 55m],[],[33 Seats available\n16 Window],[2.4]
162,Cooch Behar (West Bengal) to Farakka Bus,https://www.redbus.in/bus-tickets/cooch-behar-...,[NBSTC-COB-BHP-R1-19:00],[Non AC Seater 2+2],[19:00],[04:30],[09h 30m],[367],[22 Seats available\n9 Window],[4.9]


In [45]:
#bus-7 Extract Data from BSRTC Bihar state road transport corporation (BSRTC)


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/north-bengal-state-transport-corporation')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_bsrroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            bsr_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            bsr_urls = [element.get_attribute("href") for element in bsr_elements]
            all_bsrroute_urls.extend(bsr_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_bsrroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    bsr= pd.DataFrame(bus_details)
    print(bsr)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Cooch Behar (West Bengal) to Kolkata Bus
Fa

In [46]:
bsr

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[NBSTC-KOL-BLG-R1-19:30],[NON A/C Seater Push Back (2+2)],[19:45],[07:15],[11h 30m],[367],[33 Seats available\n15 Window],[3.8]
1,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Express Line],[Volvo 9600 A/C Seater/Sleeper (2+1)],[19:45],[06:15],[10h 30m],[699],[10 Seats available\n3 Window],[4.6]
2,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Shyamoli Paribahan Pvt Ltd],[Scania Multi-Axle AC Semi Sleeper (2+2)],[20:00],[06:15],[10h 15m],[700],[34 Seats available\n13 Window],[4.0]
3,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Rubi Paribahan ],[Volvo 9600 A/C Seater/Sleeper (2+1)],[20:30],[05:00],[08h 30m],[749],[25 Seats available],[4.0]
4,Kolkata to Balurghat Bus,https://www.redbus.in/bus-tickets/kolkata-to-b...,[Sonar Tori],[A/C Seater / Sleeper (2+2)],[19:45],[05:30],[09h 45m],[],[32 Seats available\n16 Window],[3.2]
...,...,...,...,...,...,...,...,...,...,...
154,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,[NBSTC-RAI-KOL-R1-19:30],[NON A/C Seater Push Back (2+2)],[19:30],[04:15],[08h 45m],[278],[35 Seats available\n17 Window],[3.3]
155,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,[Lokenath Bus Service],[NON A/C Seater / Sleeper (2+2)],[19:45],[04:10],[08h 25m],[500],[11 Seats available],[3.0]
156,Raiganj to Ranaghat Bus,https://www.redbus.in/bus-tickets/raiganj-to-r...,[Aradhana Bus Service],[NON A/C Seater / Sleeper (2+2)],[19:15],[03:10],[07h 55m],[],[33 Seats available\n16 Window],[2.4]
157,Cooch Behar (West Bengal) to Farakka Bus,https://www.redbus.in/bus-tickets/cooch-behar-...,[NBSTC-COB-BHP-R1-19:00],[Non AC Seater 2+2],[19:00],[04:30],[09h 30m],[367],[22 Seats available\n9 Window],[4.9]


In [47]:
#bus-8 Extract Data from South Bengal State Transport Corporation (SBSTC)



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_sbtcroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            sbtc_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            sbtc_urls = [element.get_attribute("href") for element in sbtc_elements]
            all_sbtcroute_urls.extend(sbtc_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_sbtcroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_link':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    sbtc= pd.DataFrame(bus_details)
    print(sbtc)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Asansol (West Bengal) to Kolkata Bus
succes

In [48]:
sbtc

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KOLKATA - 15:35 (HOWRAH DEPOT...,[Non AC Seater (2+3)],[15:35],[20:05],[04h 30m],[180],[51 Seats available\n19 Window],[1.8]
1,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KARUNAMOYEE - 16:10 (BEL DEPO...,[Non AC Seater (2+3)],[16:10],[21:10],[05h 00m],[185],[51 Seats available\n19 Window],[3.3]
2,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KOLKATA - 16:30 (HOWRAH DEPOT...,[Non AC Seater (2+3)],[16:30],[21:00],[04h 30m],[180],[52 Seats available\n20 Window],[2.8]
3,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[SBSTC-ASANSOL - KARUNAMOYEE - 16:40 (ASANSOL ...,[Non AC Seater (2+3)],[16:40],[22:00],[05h 20m],[185],[50 Seats available\n18 Window],[2.8]
4,Asansol (West Bengal) to Kolkata Bus,https://www.redbus.in/bus-tickets/asansol-to-k...,[Express Line (Karunamoyee)],[Volvo 9600 A/C Seater (2+2)],[15:20],[20:25],[05h 05m],[524],[26 Seats available\n8 Window],[4.7]
...,...,...,...,...,...,...,...,...,...,...
217,Siliguri to Malda Bus,https://www.redbus.in/bus-tickets/siliguri-to-...,[Shyamoli Paribahan Pvt Ltd],[Volvo 9600 A/C Seater/Sleeper (2+1)],[18:30],[00:10],[05h 40m],[1200],[41 Seats available\n6 Single],[3.9]
218,Siliguri to Malda Bus,https://www.redbus.in/bus-tickets/siliguri-to-...,[EASY RIDE],[Volvo 9600 Multi-Axle A/C Sleeper (2+1)],[18:45],[23:55],[05h 10m],[1500],[22 Seats available\n5 Single],[4.7]
219,Siliguri to Malda Bus,https://www.redbus.in/bus-tickets/siliguri-to-...,[Greenline],[Volvo 9600 Multi-Axle A/C Sleeper (2+1)],[19:00],[23:59],[04h 59m],[1400],[41 Seats available],[4.3]
220,Kolkata to Suri Bus,https://www.redbus.in/bus-tickets/kolkata-to-suri,[SBSTC-KOLKATA - SURI - 16: 30 - (Suri Depot) ...,[Non AC Seater (2+3)],[16:30],[21:30],[05h 00m],[190],[52 Seats available\n20 Window],[3.6]


In [49]:
#bus-9 Extract Data from jksrtc jammu kashmir road


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/jksrtc')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_jksrtcroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            jksrtc_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            jksrtc_urls = [element.get_attribute("href") for element in jksrtc_elements]
            all_jksrtcroute_urls.extend(jksrtc_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_jksrtcroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_url':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    jksrtc= pd.DataFrame(bus_details)
    print(jksrtc)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Failed to navigate to page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Haridwar to Jammu Bus
Failed to click expand button: Message: 
Stacktrace:
	GetHandleVerifier [0x00007F

In [50]:
jksrtc

,route_name,route_url,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Haridwar to Jammu Bus,https://www.redbus.in/bus-tickets/haridwar-to-...,[SHRI KRISHNA TRAVELS (JAI SHREE GANESH YATRA ...,[VE A/C Seater / Sleeper (2+1)],[18:20],[07:00],[12h 40m],[3000],[4 Seats available\n2 Window],[3.8]
1,Haridwar to Jammu Bus,https://www.redbus.in/bus-tickets/haridwar-to-...,[City Land Travels],[A/C Seater/Sleeper (2+1)],[17:31],[08:00],[14h 29m],[],[16 Seats available\n2 Single],[4.1]
2,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Moon Light Travels Katra],[A/C Sleeper (2+1)],[21:15],[04:30],[07h 15m],[],[14 Seats available\n4 Single],[4.4]
3,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Laxmi holidays],[Bharat Benz A/C Seater /Sleeper (2+1)],[23:05],[06:00],[06h 55m],[],[15 Seats available\n3 Single],[4.3]
4,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Zimindara Travels],[A/C Seater / Sleeper (2+1)],[22:30],[05:30],[07h 00m],[299],[24 Seats available\n12 Window],[4.2]
5,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[AdMoon Light Travels Katra],[A/C Seater / Sleeper (2+2)],[22:30],[05:15],[06h 45m],[],[40 Seats available\n16 Window],[3.4]
6,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Harikesh Tour N Travels],[A/C Sleeper (2+1)],[21:00],[04:30],[07h 30m],[],[12 Seats available\n5 Single],[4.0]
7,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Laxmi holidays],[Volvo 9600 Multi-Axle A/C Sleeper (2+1)],[23:50],[07:00],[07h 10m],[],[22 Seats available\n6 Single],[4.4]
8,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[Baru travels],[A/C Seater / Sleeper (2+2)],[20:45],[04:15],[07h 30m],[400],[19 Seats available\n9 Window],[3.9]
9,Chandigarh to Jammu Bus,https://www.redbus.in/bus-tickets/chandigarh-t...,[AdRadhe Travels],[Bharat Benz A/C Sleeper (2+1)],[21:20],[04:45],[07h 25m],[],[29 Seats available],[3.8]


In [51]:
#bus-10 Extract Data from jAssam State Transport Corporation (ASTC)



from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)  # 10 seconds timeout for explicit waits

try:
    # Open the main website
    driver.get('https://www.redbus.in/online-booking/astc/?utm_source=rtchometile')

    # Wait for the body to load
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    
    # Scroll down to ensure all elements are loaded (optional)
    for _ in range(3):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Initialize an empty list to store the URLs of detail pages
    all_astcroute_urls = []

    # Adjusted to navigate through 4 pages
    page_number = 1
    while True:
        try:
            # Wait for pagination container to load
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))

            # Find the next page button for current page_number
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class, "DC_117_pageTabs") and text()="{page_number+1}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
            time.sleep(2)
            next_page_button.click()
            print(f'Navigated to page {page_number + 1}')
            time.sleep(5)  # Wait for the new page to load

            # Extract URLs from current page
            astc_elements = driver.find_elements(By.XPATH, '//a[@class="route"]')
            astc_urls = [element.get_attribute("href") for element in astc_elements]
            all_astcroute_urls.extend(astc_urls)

            page_number += 1

        except Exception as e:
            print(f"Failed to navigate to page {page_number + 1}: {e}")
            break  # Exit loop if next page button not found or any other error

    # Initialize an empty list to store bus details
    bus_details = []

    # Iterate through each URL and extract data from detail pages
    for url in all_astcroute_urls:
        try:
            driver.get(url)
            driver.maximize_window()
            time.sleep(5)  # Adjust if needed  

            # Scroll down the page until no more content is loaded
            old_page_height = driver.execute_script("return document.body.scrollHeight")
            
            route_name_element = driver.find_element(By.XPATH, '//h1[@class="D136_h1"]')
            route_name = route_name_element.text.strip()
            print(f"Extracted Route Name: {route_name}")
            try:
             button = wait.until(EC.element_to_be_clickable((By.XPATH, '//div[@class="button" and text() ="View Buses"]')))
             #driver.execute_script("arguments[0].scrollIntoView(true);", button)  # Scroll into view if necessary  # Optional: Wait a bit before clicking
             button.click()
             driver.execute_script("arguments[0].scrollIntoView(true);", button)   
             time.sleep(2)  # Wait after clicking, adjust as necessary
             print("success")
            except Exception as e:
               print(f"Failed to click expand button: {e}")

            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                new_page_height = driver.execute_script("return document.body.scrollHeight")
                if new_page_height == old_page_height:
                    break
                old_page_height = new_page_height
            
            
            # Wait for bus items to load
            bus_container = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-item")))

            for bus in bus_container:
                try:
                    # Find and extract bus details
                    bus_name = bus.find_elements(By.XPATH, ".//div[@class='travels lh-24 f-bold d-color']")
                    bus_names = [i.text for i in bus_name]
                    bus_type = bus.find_elements(By.XPATH, ".//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                    bus_types = [i.text for i in bus_type]
                    starting_time = bus.find_elements(By.XPATH, ".//div[@class='dp-time f-19 d-color f-bold']")
                    starting_times = [i.text for i in starting_time]
                    ending_time = bus.find_elements(By.XPATH, ".//div[@class='bp-time f-19 d-color disp-Inline']")
                    ending_times = [i.text for i in ending_time]
                    bus_duration = bus.find_elements(By.XPATH, ".//div[@class='dur l-color lh-24']") 
                    time_duration = [i.text for i in bus_duration] 
                    price = bus.find_elements(By.XPATH, ".//span[@class='f-19 f-bold']")
                    prices = [i.text for i in price]
                    seat = bus.find_elements(By.XPATH, ".//div[@class='column-eight w-15 fl']")
                    seats = [i.text for i in seat]  
                    star_rating = bus.find_elements(By.XPATH, ".//div[@class='rating-sec lh-24']")
                    star_ratings = [i.text for i in star_rating]   

                    # Append bus details to bus_details list
                    bus_details.append({
                        'route_name': route_name,
                        'route_url':url,
                        'bus_name': bus_names,
                        'bus_type': bus_types,
                        'departing_time': starting_times,
                        'reaching_time': ending_times, 
                        'bus_duration': time_duration,
                        'price': prices,
                        'seat': seats,
                        'star_rating': star_ratings
                    })

                except Exception as e:
                    print(f"Failed to extract details from bus: {e}")
                    continue  # Skip to the next bus element if extraction fails

        except Exception as e:
            print(f"Failed to process URL {url}: {e}")
            continue  # Skip to the next URL if processing fails

    # Create a DataFrame from bus_details list
    astc= pd.DataFrame(bus_details)
    print(astc)

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Close the WebDriver
    driver.quit()

Navigated to page 2
Navigated to page 3
Navigated to page 4
Navigated to page 5
Failed to navigate to page 6: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7E128EEB2+31554]
	(No symbol) [0x00007FF7E1207EE9]
	(No symbol) [0x00007FF7E10C872A]
	(No symbol) [0x00007FF7E1118434]
	(No symbol) [0x00007FF7E111853C]
	(No symbol) [0x00007FF7E115F6A7]
	(No symbol) [0x00007FF7E113D06F]
	(No symbol) [0x00007FF7E115C977]
	(No symbol) [0x00007FF7E113CDD3]
	(No symbol) [0x00007FF7E110A33B]
	(No symbol) [0x00007FF7E110AED1]
	GetHandleVerifier [0x00007FF7E1598B2D+3217341]
	GetHandleVerifier [0x00007FF7E15E5AF3+3532675]
	GetHandleVerifier [0x00007FF7E15DB0F0+3489152]
	GetHandleVerifier [0x00007FF7E133E786+750614]
	(No symbol) [0x00007FF7E121376F]
	(No symbol) [0x00007FF7E120EB24]
	(No symbol) [0x00007FF7E120ECB2]
	(No symbol) [0x00007FF7E11FE17F]
	BaseThreadInitThunk [0x00007FFB7E9F257D+29]
	RtlUserThreadStart [0x00007FFB7F58AF28+40]

Extracted Route Name: Dhekiajuli to Guwahati Bus
Failed to click 

In [52]:
astc

,route_name,route_url,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating
0,Tezpur to Dibrugarh Bus,https://www.redbus.in/bus-tickets/tezpur-to-di...,[Aniruddha Travels],[NON A/C Seater/ Sleeper (2+1)],[20:00],[04:00],[08h 00m],[550],[2 Seats available],[3.5]
1,Tezpur to Dibrugarh Bus,https://www.redbus.in/bus-tickets/tezpur-to-di...,[Puja Transport],[NON AC Seater / Sleeper 2+1],[20:00],[04:30],[08h 30m],[550],[1 Seat available],[3.5]
2,Tezpur to Dibrugarh Bus,https://www.redbus.in/bus-tickets/tezpur-to-di...,[Puja Transport],[Non AC Seater (2+2)],[20:00],[05:00],[09h 00m],[550],[5 Seats available\n2 Window],[2.4]
3,Tezpur to Dibrugarh Bus,https://www.redbus.in/bus-tickets/tezpur-to-di...,[J.S. Travels],[Non AC Seater (2+1)],[21:00],[04:00],[07h 00m],[550],[32 Seats available\n20 Window],[2.3]
4,Tezpur to Dibrugarh Bus,https://www.redbus.in/bus-tickets/tezpur-to-di...,[Neha Travels],[NON A/C Seater (2+2)],[22:40],[07:00],[08h 20m],[650],[1 Seat available],[2.1]
...,...,...,...,...,...,...,...,...,...,...
89,Tezpur to Tinsukia Bus,https://www.redbus.in/bus-tickets/tezpur-to-ti...,[Neha Travels],[NON A/C Seater (2+2)],[22:40],[08:00],[09h 20m],[650],[1 Seat available],[1.4]
90,Dibrugarh to Bokakhat Bus,https://www.redbus.in/bus-tickets/dibrugarh-to...,[Baikuntha Transport Service],[Bharat Benz A/C Seater /Sleeper (2+1)],[19:30],[01:00],[05h 30m],[],[27 Seats available\n15 Window],[3.9]
91,Dibrugarh to Bokakhat Bus,https://www.redbus.in/bus-tickets/dibrugarh-to...,[Purple Wings Coaches],[A/C Seater / Sleeper (2+1)],[19:55],[01:15],[05h 20m],[],[12 Seats available\n1 Single],[3.4]
92,Dibrugarh to Bokakhat Bus,https://www.redbus.in/bus-tickets/dibrugarh-to...,[Shree Ram Travels],[Bharat Benz A/C Seater /Sleeper (2+1)],[20:45],[01:40],[04h 55m],[],[8 Seats available\n2 Single],[4.8]


In [75]:
all_bus = pd.concat([df, hrtc, wbtc, ctu, pepsu, nbs, bsr, sbtc, jksrtc, astc], axis=0, ignore_index=True)


In [85]:
csv_filename = 'redbus.csv'
all_bus.to_csv(csv_filename, index=False)


In [8]:
#all_bus =pd.read_csv('C:\\Users\\Madhu\\Downloads\\all_bus.csv')

In [54]:
all_bus_cleaned = all_bus.where(pd.notnull(all_bus), None)


In [55]:
all_bus_cleaned

,route_name,route_link,bus_name,bus_type,departing_time,reaching_time,bus_duration,price,seat,star_rating,route_lin,route_url
0,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[Kadamba Transport Corporation Limited (KTCL) ...,[Non AC Seater 2+2],[19:00],[02:30],[07h 30m],[420],[24 Seats available\n9 Window],[3.9],None,None
1,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[IntrCity SmartBus],[AC Sleeper (2+1)],[16:30],[22:30],[06h 00m],[550],[9 Seats available\n4 Single],[4.1],None,None
2,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[IntrCity SmartBus],[AC Sleeper (2+1)],[18:20],[00:35],[06h 15m],[600],[16 Seats available\n4 Single],[4.3],None,None
3,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[IntrCity SmartBus],[AC Sleeper (2+1)],[18:30],[00:45],[06h 15m],[599],[13 Seats available\n3 Single],[4.5],None,None
4,Goa to Kolhapur(Maharashtra) Bus,https://www.redbus.in/bus-tickets/goa-to-kolha...,[Amrutyog Travels],[A/C Sleeper (2+1)],[19:00],[02:30],[07h 30m],[555],[20 Seats available\n4 Single],[4.4],None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2168,Tezpur to Tinsukia Bus,None,[Neha Travels],[NON A/C Seater (2+2)],[22:40],[08:00],[09h 20m],[650],[1 Seat available],[1.4],None,https://www.redbus.in/bus-tickets/tezpur-to-ti...
2169,Dibrugarh to Bokakhat Bus,None,[Baikuntha Transport Service],[Bharat Benz A/C Seater /Sleeper (2+1)],[19:30],[01:00],[05h 30m],[],[27 Seats available\n15 Window],[3.9],None,https://www.redbus.in/bus-tickets/dibrugarh-to...
2170,Dibrugarh to Bokakhat Bus,None,[Purple Wings Coaches],[A/C Seater / Sleeper (2+1)],[19:55],[01:15],[05h 20m],[],[12 Seats available\n1 Single],[3.4],None,https://www.redbus.in/bus-tickets/dibrugarh-to...
2171,Dibrugarh to Bokakhat Bus,None,[Shree Ram Travels],[Bharat Benz A/C Seater /Sleeper (2+1)],[20:45],[01:40],[04h 55m],[],[8 Seats available\n2 Single],[4.8],None,https://www.redbus.in/bus-tickets/dibrugarh-to...


In [18]:
import pandas as pd

In [74]:
import pymysql

connection = pymysql.connect(host='127.0.0.1',
                             user='root',
                             password='Madhu@29',
                             database='redbus',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS bus_values5(
    route_name TEXT,
    route_link TEXT,
    bus_names TEXT,
    bus_types TEXT,
    departing_time TIME,
    reaching_time TIME,
    bus_duration TEXT,
    prices DECIMAL(10, 2),
    seats INT,
    star_ratings FLOAT
)
"""

try:
    cursor.execute(create_table_query)
    print("Table 'bus_values5' created successfully.")
except pymysql.Error as e:
    print("Error creating table:", e)

insert_query = """
INSERT INTO bus_values5
(route_name, route_link, bus_names, bus_types, departing_time, reaching_time, bus_duration, prices, seats, star_ratings) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

bus_values5 = all_bus_cleaned.values.tolist()  # Assuming all_bus_cleaned is a pandas DataFrame

for row in bus_values5:
    try:
        if len(row[7]) > 0:  # Check if row[7] has elements
            prices = float(row[7][0].strip("[]"))  # Convert prices to float
        else:
            prices = None  # Handle case where prices are not available

        seats = int(row[8])  # Convert seats to int

        if len(row[9]) > 0:  # Check if row[9] has elements
            star_ratings = float(row[9][0].strip("[]"))  # Convert star_ratings to float
        else:
            star_ratings = None  # Handle case where star_ratings are not available

        cursor.execute(insert_query, (
            row[0],  # route_name
            row[1],  # route_link
            row[2],  # bus_names
            row[3],  # bus_types
            row[4],  # departing_time
            row[5],  # reaching_time
            row[6],  # bus_duration
            prices,  # prices
            seats,  # seats
            star_ratings  # star_ratings
        ))

        print("Inserted row successfully:", row[0])
    except pymysql.Error as e:
        print("Error inserting data:", e)

connection.commit()

Table 'bus_values5' created successfully.
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bus
Inserted row successfully: Goa to Kolhapur(Maharashtra) Bu